In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [2]:
#Use this file path for pc
filepath = r"D:\gradschool\Capstone"
#Use this file path for mac 
# filepath = r"/Users/jacksonjoyner/Desktop/MSBA/Capstone/UseThisData/"
os.chdir(filepath)

In [3]:
playerdf = pd.read_csv("PCATest1.csv")
playerdf = playerdf.rename(columns ={"total cs" : "total_cs"})
playerdf["total_cs"]

0        360
1        155
2        411
3        358
4         37
        ... 
92595    281
92596    208
92597    250
92598    315
92599     41
Name: total_cs, Length: 92600, dtype: int64

Part of this data was incorrect, so I corrected it in excel. For KDA, the calculation is (Kills + Assists)/Deaths. If Deaths is 0, you get an infinite number, so I changed it in those instances to be (kills + Assists) / 1. This is common practice in industry. 

From testing, this is the most accurate model I can create

In [4]:
playerdf

,league,year,split,date,side,playername,position,teamname,kills,deaths,...,D%,earnedgoldshare,damageshare,cspm,damagetakenperminute,earnedgold,earned gpm,PCA15,PCAShare,PCApm
0,EU LCS,2014,Spring,3/28/2014 16:14,Blue,fredy122,top,SK Gaming,0,3,...,0.136364,0.233888,0.266095,8.1509,926.4000,8884,201.1472,1.071165,0.851092,0.192018
1,EU LCS,2014,Spring,3/28/2014 16:14,Blue,Svenskeren,jng,SK Gaming,4,4,...,0.181818,0.168887,0.174089,3.5094,733.4717,6415,145.2453,-1.157427,-0.536872,-1.329622
2,EU LCS,2014,Spring,3/28/2014 16:14,Blue,Jesiz,mid,SK Gaming,1,3,...,0.136364,0.244866,0.284035,9.3057,576.5434,9301,210.5887,1.115521,1.102367,0.530580
3,EU LCS,2014,Spring,3/28/2014 16:14,Blue,CandyPanda,bot,SK Gaming,2,5,...,0.227273,0.254186,0.192568,8.1057,685.9698,9655,218.6038,-0.831752,0.566422,0.338479
4,EU LCS,2014,Spring,3/28/2014 16:14,Blue,nRated,sup,SK Gaming,1,7,...,0.318182,0.098173,0.083214,0.8377,528.8151,3729,84.4302,-2.931761,-1.982099,-2.462070
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92595,LCS,2021,Spring,4/3/2021 21:32,Red,Alphari,top,Team Liquid,1,2,...,0.117647,0.220126,0.224882,8.6550,442.2690,7041,216.8686,1.414158,0.404612,0.443739
92596,LCS,2021,Spring,4/3/2021 21:32,Red,Santorin,jng,Team Liquid,1,4,...,0.235294,0.181831,0.093968,6.4066,773.9322,5816,179.1376,0.906147,-0.951782,-0.388794
92597,LCS,2021,Spring,4/3/2021 21:32,Red,Jensen,mid,Team Liquid,3,5,...,0.294118,0.246104,0.327698,7.7002,707.8029,7872,242.4641,1.724921,1.423113,0.463075
92598,LCS,2021,Spring,4/3/2021 21:32,Red,Tactical,bot,Team Liquid,1,3,...,0.176471,0.270988,0.295341,9.7023,284.8460,8668,266.9815,0.601493,1.479895,1.123052


In [5]:
variables = ['kills','deaths','assists','PCA15','PCAShare','PCApm']
x = playerdf[variables].to_numpy() 
y = playerdf["result"]

In [14]:
#https://www.datasklr.com/ols-least-squares-regression/multicollinearity
model = sm.OLS(y, sm.add_constant(x)).fit()
Y_pred = model.predict(sm.add_constant(x))
print_model = model.summary()
print(print_model)

                            OLS Regression Results                            
Dep. Variable:                 result   R-squared:                       0.623
Model:                            OLS   Adj. R-squared:                  0.623
Method:                 Least Squares   F-statistic:                 2.546e+04
Date:                Wed, 30 Mar 2022   Prob (F-statistic):               0.00
Time:                        14:50:42   Log-Likelihood:                -22092.
No. Observations:               92600   AIC:                         4.420e+04
Df Residuals:                   92593   BIC:                         4.426e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.3208      0.003    124.265      0.0

In [12]:
Y_pred

array([0.72581718, 0.80069344, 0.83278929, ..., 0.35309204, 0.88367163,
       0.56592162])

In [18]:
playerdf["predscore"] = Y_pred
playerdf.to_csv("LogitModelPreds.csv", index = False)

In [17]:
playerdf[5:10]

,league,year,split,date,side,playername,position,teamname,kills,deaths,...,earnedgoldshare,damageshare,cspm,damagetakenperminute,earnedgold,earned gpm,PCA15,PCAShare,PCApm,predscore
5,EU LCS,2014,Spring,3/28/2014 16:14,Red,Wickd,top,Alliance,2,2,...,0.220601,0.126431,9.1019,931.9245,12764,288.9962,1.702089,-0.281326,1.188248,1.070121
6,EU LCS,2014,Spring,3/28/2014 16:14,Red,Shook,jng,Alliance,5,2,...,0.167974,0.201310,3.7811,725.8642,9719,220.0528,-0.077568,-0.356136,-0.599633,0.778856
7,EU LCS,2014,Spring,3/28/2014 16:14,Red,Froggen,mid,Alliance,9,1,...,0.250726,0.338193,9.1245,502.4830,14507,328.4604,1.560962,1.549565,1.546797,0.997557
8,EU LCS,2014,Spring,3/28/2014 16:14,Red,Tabzz,bot,Alliance,4,1,...,0.258019,0.282897,10.0981,476.9434,14929,338.0151,-0.308584,1.244492,1.846522,1.173426
9,EU LCS,2014,Spring,3/28/2014 16:14,Red,Nyph,sup,Alliance,2,2,...,0.102679,0.051168,1.4038,536.6038,5941,134.5132,-2.941316,-2.155706,-1.888853,0.738686
